In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import gc
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
df_women = pd.read_parquet('/kaggle/input/visual-taxonomy/category_attributes.parquet')
print(df_women.head())

              Category  No_of_attribute  \
0          Men Tshirts                5   
1               Sarees               10   
2               Kurtis                9   
3        Women Tshirts                8   
4  Women Tops & Tunics               10   

                                      Attribute_list  
0  [color, neck, pattern, print_or_pattern_type, ...  
1  [blouse_pattern, border, border_width, color, ...  
2  [color, fit_shape, length, occasion, ornamenta...  
3  [color, fit_shape, length, pattern, print_or_p...  
4  [color, fit_shape, length, neck_collar, ocassi...  


In [3]:
train_path = "/kaggle/input/visual-taxonomy/train.csv"
train_df = pd.read_csv(train_path)

In [4]:
l = df_women['Attribute_list'][2]
for item in l :
    print(item)

color
fit_shape
length
occasion
ornamentation
pattern
print_or_pattern_type
sleeve_length
sleeve_styling


In [5]:
for i in range(1,10) :
    temp = train_df.groupby(f'attr_{i}').apply(len)
    print(temp) 
    print('--------------------------------------------')
    print('\n')
    
#color, fs, length, pattern, pp

attr_1
black             7901
blue              1371
default           9268
green              924
grey               137
maroon            2597
multicolor        4076
navy blue         1140
orange              33
peach              397
pink              3132
purple             170
red               2178
same as border    1611
same as saree     5230
solid              188
white             8402
yellow            3112
dtype: int64
--------------------------------------------


attr_2
a-line            2066
boxy              2975
default           1790
fitted            5523
loose             1483
no border          313
polo              3085
regular          17222
round             3059
solid              411
straight          1165
temple border     1296
woven design      6262
zari              8542
dtype: int64
--------------------------------------------


attr_3
big border       5781
calf length      1352
crop            12418
knee length      2048
long             1682
no border    

/tmp/ipykernel_30/847025047.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp = train_df.groupby(f'attr_{i}').apply(len)
/tmp/ipykernel_30/847025047.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp = train_df.groupby(f'attr_{i}').apply(len)
/tmp/ipykernel_30/847025047.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a futur

In [10]:
df_women = train_df.loc[train_df['Category'] == 'Kurtis']
df_women.reset_index(drop=True, inplace=True)
df_women

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,25778,Kurtis,9,black,straight,knee length,daily,net,solid,solid,three-quarter sleeves,regular,NaN
1,25779,Kurtis,9,red,straight,knee length,daily,default,default,default,three-quarter sleeves,regular,NaN
2,25780,Kurtis,9,red,straight,knee length,daily,default,default,default,three-quarter sleeves,regular,NaN
3,25781,Kurtis,9,navy blue,straight,knee length,daily,default,default,default,three-quarter sleeves,regular,NaN
4,25782,Kurtis,9,black,straight,knee length,daily,default,default,default,three-quarter sleeves,regular,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6817,32596,Kurtis,9,black,NaN,NaN,daily,NaN,NaN,NaN,three-quarter sleeves,regular,NaN
6818,32597,Kurtis,9,red,NaN,NaN,daily,NaN,NaN,NaN,three-quarter sleeves,regular,NaN
6819,32598,Kurtis,9,multicolor,NaN,NaN,daily,NaN,NaN,NaN,three-quarter sleeves,regular,NaN
6820,32599,Kurtis,9,red,NaN,NaN,daily,NaN,NaN,NaN,three-quarter sleeves,regular,NaN


In [7]:
df_women.isnull().sum()

id             0
Category       0
len            0
attr_1       193
attr_2      3591
attr_3      3422
attr_4       391
attr_5      3556
attr_6      2974
attr_7      2979
attr_8       120
attr_9       131
attr_10     6822
dtype: int64

In [8]:
from keras.utils import normalize, to_categorical
pd.options.mode.chained_assignment = None
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from tqdm import tqdm
from keras.preprocessing import image
from sklearn.model_selection import train_test_split

In [11]:
import pandas as pd
import json

# Initialize a dictionary to store mappings for each attribute
mappings = {}

# Iterate through each attribute column to create and save mappings
for attr in df_women.columns[3:12]:  # Adjust column range if necessary
    # Convert column to categorical and get the categories
    c = df_women[attr].astype('category')
    df_women[attr] = c.cat.codes  # Apply cat.codes to the DataFrame
    
    # Store the mapping of integer codes to original categories
    mappings[attr] = dict(enumerate(c.cat.categories))

# Display the mappings for verification
print("Mappings:")
for key, value in mappings.items():
    print(f"{key}: {value}")

# Save mappings to /kaggle/working/ for download
with open("/kaggle/working/attribute_mappings.json", "w") as f:
    json.dump(mappings, f, indent=4)

print("\nMappings saved to /kaggle/working/attribute_mappings.json")


Mappings:
attr_1: {0: 'black', 1: 'blue', 2: 'green', 3: 'grey', 4: 'maroon', 5: 'multicolor', 6: 'navy blue', 7: 'orange', 8: 'pink', 9: 'purple', 10: 'red', 11: 'white', 12: 'yellow'}
attr_2: {0: 'a-line', 1: 'straight'}
attr_3: {0: 'calf length', 1: 'knee length'}
attr_4: {0: 'daily', 1: 'party'}
attr_5: {0: 'default', 1: 'net'}
attr_6: {0: 'default', 1: 'solid'}
attr_7: {0: 'default', 1: 'solid'}
attr_8: {0: 'short sleeves', 1: 'sleeveless', 2: 'three-quarter sleeves'}
attr_9: {0: 'regular', 1: 'sleeveless'}

Mappings saved to /kaggle/working/attribute_mappings.json


In [12]:
# df_women['attr_1'] = df_women["attr_1"].astype('category').cat.codes
# df_women['attr_2'] = df_women["attr_2"].astype('category').cat.codes
# df_women['attr_3'] = df_women["attr_3"].astype('category').cat.codes
# df_women['attr_4'] = df_women["attr_4"].astype('category').cat.codes
# df_women['attr_5'] = df_women["attr_5"].astype('category').cat.codes
# df_women['attr_6'] = df_women["attr_6"].astype('category').cat.codes
# df_women['attr_7'] = df_women["attr_7"].astype('category').cat.codes
# df_women['attr_8'] = df_women["attr_8"].astype('category').cat.codes
# df_women['attr_9'] = df_women["attr_9"].astype('category').cat.codes


y_color = to_categorical(np.array(df_women['attr_1']))
y_fs = to_categorical(np.array(df_women['attr_2']))
y_length = to_categorical(np.array(df_women['attr_3']))
y_occasion = to_categorical(np.array(df_women['attr_4']))
y_ornament = to_categorical(np.array(df_women['attr_5']))
y_pattern = to_categorical(np.array(df_women['attr_6']))
y_pp = to_categorical(np.array(df_women['attr_7']))
y_sl = to_categorical(np.array(df_women['attr_8']))
y_ss = to_categorical(np.array(df_women['attr_9']))

In [16]:
image_directory = "/kaggle/input/visual-taxonomy/train_images/"
SIZE=200
X_dataset = []  
for i in tqdm(range(df_women.shape[0])):
    prep = str(df_women['id'][i])
    prep = '0'*(6-len(prep)) + prep
    img = image.load_img(image_directory +prep+'.jpg', target_size=(SIZE,SIZE,3))
    img = image.img_to_array(img)
    img = img/255.
    X_dataset.append(img)
X = np.array(X_dataset)

100%|██████████| 6822/6822 [00:49<00:00, 137.74it/s]


In [26]:
print(np.unique(np.array(df_women['attr_1'])))

[-1  0  1  2  3  4  5  6  7  8  9 10 11 12]


In [20]:
df_women['attr_1'].unique()

array([ 0, 10,  6,  4,  2, -1,  8,  1,  9,  3, 12, 11,  5,  7], dtype=int8)

In [21]:
y_color.shape

(6822, 13)

In [ ]:
NUM_CLASSES_COLOR = len(df_women['attr_1'].unique())-1
NUM_CLASSES_FS = len(df_women['attr_2'].unique())-1
NUM_CLASSES_LENGTH = len(df_women['attr_3'].unique())-1
NUM_CLASSES_OCCASION = len(df_women['attr_4'].unique())-1
NUM_CLASSES_ORNAMENT = len(df_women['attr_5'].unique())-1
NUM_CLASSES_PATTERN = len(df_women['attr_6'].unique())-1
NUM_CLASSES_PP = len(df_women['attr_7'].unique())-1
NUM_CLASSES_SL = len(df_women['attr_8'].unique())-1
NUM_CLASSES_SS = len(df_women['attr_9'].unique())-1


In [ ]:
IMAGE_SIZE = (200, 200, 3)


# Load the base model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=IMAGE_SIZE)

# Freeze the base model
base_model.trainable = True


In [ ]:
import keras 

In [ ]:
y_combined = np.hstack((y_color,y_fs,y_length,y_occasion,y_ornament,y_pattern,
y_pp,y_sl, y_ss))

# Split into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y_combined,
    test_size=0.2,
    random_state=42
)

In [ ]:
del X
del train_df 
gc.collect()

In [ ]:

# Split the combined labels back into their respective arrays
train_color_labels = y_train[:, :NUM_CLASSES_COLOR]
train_fs_labels = y_train[:, NUM_CLASSES_COLOR:NUM_CLASSES_COLOR + NUM_CLASSES_FS]
train_length_labels = y_train[:, NUM_CLASSES_COLOR + NUM_CLASSES_FS:NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH]
train_occasion_labels = y_train[:, NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH:NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH + NUM_CLASSES_OCCASION]
train_ornament_labels = y_train[:, NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH + NUM_CLASSES_OCCASION:NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH + NUM_CLASSES_OCCASION+NUM_CLASSES_ORNAMENT]
train_pattern_labels = y_train[:,  -NUM_CLASSES_SS-NUM_CLASSES_SL-NUM_CLASSES_PP-NUM_CLASSES_PATTERN:-NUM_CLASSES_SS-NUM_CLASSES_SL-NUM_CLASSES_PP]
train_pp_labels = y_train[:, -NUM_CLASSES_SS-NUM_CLASSES_SL-NUM_CLASSES_PP:-NUM_CLASSES_SS-NUM_CLASSES_SL]
train_sl_labels = y_train[:, -NUM_CLASSES_SS-NUM_CLASSES_SL:-NUM_CLASSES_SS]
train_ss_labels = y_train[:, -NUM_CLASSES_SS:]

val_color_labels = y_val[:, :NUM_CLASSES_COLOR]
val_fs_labels = y_val[:, NUM_CLASSES_COLOR:NUM_CLASSES_COLOR + NUM_CLASSES_FS]
val_length_labels = y_val[:, NUM_CLASSES_COLOR + NUM_CLASSES_FS:NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH]
val_occasion_labels = y_val[:, NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH:NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH + NUM_CLASSES_OCCASION]
val_ornament_labels = y_val[:, NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH + NUM_CLASSES_OCCASION:NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH + NUM_CLASSES_OCCASION+NUM_CLASSES_ORNAMENT]
val_pattern_labels = y_val[:,  -NUM_CLASSES_SS-NUM_CLASSES_SL-NUM_CLASSES_PP-NUM_CLASSES_PATTERN:-NUM_CLASSES_SS-NUM_CLASSES_SL-NUM_CLASSES_PP]
val_pp_labels = y_val[:, -NUM_CLASSES_SS-NUM_CLASSES_SL-NUM_CLASSES_PP:-NUM_CLASSES_SS-NUM_CLASSES_SL]
val_sl_labels = y_val[:, -NUM_CLASSES_SS-NUM_CLASSES_SL:-NUM_CLASSES_SS]
val_ss_labels = y_val[:, -NUM_CLASSES_SS:]


In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 145

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
# Create a model with five output heads for each attribute
inputs = layers.Input(shape=IMAGE_SIZE)
x = base_model(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)

# Define output layers for each attribute
output_color = layers.Dense(NUM_CLASSES_COLOR, activation='softmax', name='color')(x)
output_fs = layers.Dense(NUM_CLASSES_FS, activation='softmax', name='fs')(x)
output_length = layers.Dense(NUM_CLASSES_LENGTH, activation='softmax', name='length')(x)
output_occasion = layers.Dense(NUM_CLASSES_OCCASION, activation='softmax', name='occasion')(x)
output_ornament = layers.Dense(NUM_CLASSES_ORNAMENT, activation='softmax', name='ornament')(x)
output_pattern = layers.Dense(NUM_CLASSES_PATTERN, activation='softmax', name='pattern')(x)
output_pp = layers.Dense(NUM_CLASSES_PP, activation='softmax', name='pp')(x)
output_sl = layers.Dense(NUM_CLASSES_SL, activation='softmax', name='sl')(x)
output_ss = layers.Dense(NUM_CLASSES_SS, activation='softmax', name='ss')(x)


# Define the model
model = models.Model(inputs=inputs, outputs=[output_color, output_fs, output_length,output_occasion,output_ornament, output_pattern, output_pp,output_sl,output_ss])

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=2*1e-5),
    loss={
        'color': 'categorical_crossentropy',
        'fs': 'categorical_crossentropy',
        'length': 'categorical_crossentropy',
        'occasion':'categorical_crossentropy',
        'ornament':'categorical_crossentropy',
        'pattern': 'categorical_crossentropy',
        'pp': 'categorical_crossentropy',
        'sl': 'categorical_crossentropy',
        'ss': 'categorical_crossentropy'
    },
    metrics=['accuracy', 'accuracy', 'accuracy', 'accuracy', 'accuracy','accuracy', 'accuracy', 'accuracy','accuracy']
)

# Summary of the model
model.summary()

In [ ]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=45,
    width_shift_range=0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

In [ ]:
class MultiLabelDataGenerator(Sequence):
    def __init__(self, images, labels, batch_size, datagen):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.datagen = datagen
        self.indices = np.arange(len(images))

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_images = self.images[batch_indices]
        batch_labels = {key: label[batch_indices] for key, label in self.labels.items()}
        
        # Apply data augmentation
        batch_images = np.array([self.datagen.random_transform(img) for img in batch_images])

        return batch_images, batch_labels

In [ ]:
train_labels = {
    'color': train_color_labels,
    'fs': train_fs_labels,
    'length': train_length_labels,
    'occasion':train_occasion_labels, 
    'ornament':train_ornament_labels,
    'pattern': train_pattern_labels,
    'pp': train_pp_labels, 
    'sl': train_sl_labels,
    'ss': train_ss_labels
}
batch_size = 32
train_generator = MultiLabelDataGenerator(X_train, train_labels, batch_size, train_datagen)

In [ ]:
checkpoint_filepath = '/kaggle/working/checkpoint.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_color_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history = model.fit(
    train_generator,  # Image data
    validation_data=(X_val, {
    'color': val_color_labels,
    'fs': val_fs_labels,
    'length': val_length_labels,
    'occasion':val_occasion_labels, 
    'ornament':val_ornament_labels,
    'pattern': val_pattern_labels,
    'pp': val_pp_labels, 
    'sl': val_sl_labels,
    'ss': val_ss_labels
}),
    epochs=50,
    callbacks=[model_checkpoint_callback])

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'history' is the output from model.fit()
# Example: history = model.fit(...)
#color, fs, length, pattern, pp
# Plotting the accuracy
plt.figure(figsize=(12, 6))

# Plot training accuracy
plt.plot(history.history['color_accuracy'], label='Color Train Accuracy', color='blue')
plt.plot(history.history['length_accuracy'], label='Length Train Accuracy', color='orange')
plt.plot(history.history['pp_accuracy'], label='PP Train Accuracy', color='green')
plt.plot(history.history['pattern_accuracy'], label='Pattern Train Accuracy', color='red')
plt.plot(history.history['fs_accuracy'], label='FS Train Accuracy', color='purple')

# Plot validation accuracy
plt.plot(history.history['val_color_accuracy'], label='Color Validation Accuracy', linestyle='dashed', color='blue')
plt.plot(history.history['val_length_accuracy'], label='Length Validation Accuracy', linestyle='dashed', color='orange')
plt.plot(history.history['val_pp_accuracy'], label='PP Validation Accuracy', linestyle='dashed', color='green')
plt.plot(history.history['val_pattern_accuracy'], label='Pattern Train Accuracy',linestyle='dashed', color='red')
plt.plot(history.history['val_fs_accuracy'], label='FS Train Accuracy',linestyle='dashed', color='purple')
# Adding labels and title
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
os.chdir(r'/kaggle/working')

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink(r'/kaggle/working/checkpoint.model.keras')